In [809]:
import pandas as pd
import numpy as np
import datetime as dt

Lectura inicial

In [810]:
eventsData = pd.read_csv(r"C:\Users\brian\Documents\GitHub\TP2DATOS\Eventos(+completo).csv",low_memory = False)


In [811]:
eventsData = eventsData.drop("Unnamed: 0", axis=1)


Feature con la cantidad de static page por usuario

In [812]:
staticPageData = pd.DataFrame({'cantidadDeStaticPage' :eventsData.groupby(['person','staticpage']).size()})
staticPageData.reset_index(inplace = True)
staticPageData = staticPageData.groupby('person').agg({'cantidadDeStaticPage':'sum'}).reset_index()


Feature con la cantidad de veces que entra a un url de compra

In [813]:
#No funciono
auxiliar = pd.DataFrame({'cantidad_de_Url_de_compra' :eventsData.groupby(['person','url']).size()})
auxiliar = auxiliar.reset_index()
auxiliar["url"] = auxiliar['url'].map(lambda x: x.split('/')[1])
auxiliar = auxiliar.groupby('person').agg({'cantidad_de_Url_de_compra':'sum'}).reset_index()


Feature con la cantidad de busquedas que realiza un usuario por termino

In [814]:
#No aporta mucho
searchData = pd.DataFrame({'Cantidad_De_Busquedas_Por_Termino' :eventsData.groupby(['search_term','person','event']).size()})
searchData = searchData.sort_values(by ='Cantidad_De_Busquedas_Por_Termino',ascending= False )
searchData.reset_index(inplace = True)
searchData.set_index('person',inplace = True)
searchData = searchData.drop('event',axis=1)
searchData.reset_index(inplace=True)


Prototipos

In [815]:
#No aporta mucho
conversionData = eventsData[eventsData['event'].map(lambda x: ( x== 'conversion'))]
conversionData.set_index('person', inplace = True)
conversionData = conversionData.loc[:,['model','condition']]


In [816]:
#No sirvio
dataF = pd.DataFrame({'cantidad_de_Url_de_compra' :eventsData.groupby(['person','skus']).size()})
dataF = dataF.reset_index()
dataF['sku_multiple']= dataF['skus'].map(lambda x: len(x.split(',')))
dataF = dataF.groupby('person').agg({'sku_multiple':'sum'}).reset_index()

Feature que usa searchData y conversionData. Analiza con la cantidad de busquedas por termino y la cantidad de veces que compro lo que busco

In [817]:
#Aumento en 2%
dataSearchAndSelling = pd.merge(searchData,conversionData,on="person",how="inner")
dataSearchAndSelling
dataSearchAndSelling["model"] = dataSearchAndSelling["model"].fillna("Desconocido")
dataSearchAndSelling["marca"] = dataSearchAndSelling["model"].map(lambda x:"apple" if(x.split(" ")[0] in ["iphone","ipad"]) else x.split(" ")[0])
dataSearchAndSelling["search_term"] = dataSearchAndSelling['search_term'].map(lambda x: 'samsung' if x.split(" ")[0].lower() in ['s6','s8','s7','samsung'] else x.split(' ')[0])
dataSearchAndSelling["search_term"] = dataSearchAndSelling['search_term'].map(lambda x: 'motorola' if x.split(" ")[0].lower() in ['motorola','moto'] else x.split(' ')[0])
dataSearchAndSelling["search_term"] = dataSearchAndSelling['search_term'].map(lambda x: 'iphone' if x.split(" ")[0].lower() in ['iphone'] else x.split(' ')[0])
dataSearchAndSelling['search_and_purchase'] = dataSearchAndSelling['search_term']==(dataSearchAndSelling['marca'])
dataSearchAndSelling["search_and_purchase"] = dataSearchAndSelling['search_and_purchase'].map(lambda x: 1 if x == True else 0)
dataSearchAndSelling = dataSearchAndSelling.drop(['search_term','condition','model','marca'],axis = 1)
dataSearchAndSelling = dataSearchAndSelling.groupby('person').agg({'Cantidad_De_Busquedas_Por_Termino':'sum', 'search_and_purchase':'sum'}).reset_index()


Feature que cuenta la cantidad de sku repetidos por usuario

In [818]:
#Aporto poco
skuData = pd.DataFrame({'cantidad_De_Compras_De_Mismo_Sku' :eventsData.groupby(['sku','event','person']).size()})
skuData = skuData.reset_index()
skuData = skuData[skuData['event'].map(lambda x: ( x== 'conversion'))]
skuData = skuData.drop(['event'],axis=1)
skuData = skuData.groupby('person').agg({'sku':'first', 'cantidad_De_Compras_De_Mismo_Sku':'sum' }).reset_index()

Inicializacion de dataframes que se usaran despues

In [819]:
geoData = eventsData[["person","event","city","region","country"]]

In [820]:
deviceUsersData = eventsData[["person","browser_version","operating_system_version","screen_resolution","device_type"]]

In [821]:
productData = eventsData[["person","event","sku","model","condition","storage","skus"]]

In [822]:
infoUserData = eventsData[["person","timestamp","search_term","staticpage","campaign_source","search_engine","channel","new_vs_returning"]]

In [823]:
timeData = eventsData[["person","timestamp","Parte del dia","event"]]

In [824]:
personColor = eventsData[["person","event","color"]]

Busco crear nuevo features 

temporales

In [825]:
users_partedeldia = timeData[["person","Parte del dia"]]

In [826]:
users_partedeldia = pd.get_dummies(users_partedeldia, columns=["Parte del dia"])

In [827]:
users_partedeldia = users_partedeldia.groupby("person",as_index = False).sum()

In [828]:
#Funcion que pasa a formato datetime las fechas que recibimos.
def datetime(date,format_string):
    return pd.to_datetime(date,format= format_string,errors='coerce')

timeData["timestamp"]= datetime(timeData["timestamp"],"%Y-%m-%dT%H:%M:%S")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Mapeado de las fechas

In [829]:
timeData["year"] = timeData["timestamp"].map(lambda x: x.year)
timeData["month"] = timeData["timestamp"].map(lambda x: x.month)
timeData["day"] = timeData["timestamp"].map(lambda x: x.day)
timeData["hour"] = timeData["timestamp"].map(lambda x: x.hour)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

Feature que analiza los ultimos 15 dias

In [830]:
last_15days = timeData[["person","event","month","day"]]

In [831]:
last_15days = last_15days[last_15days["month"] == 5]

In [832]:
last_15days = last_15days[last_15days["day"] > 14]
last_15days = last_15days[["person","event"]]


In [833]:
last_15days["count_activity"] = 1

In [834]:
last_15days = pd.get_dummies(last_15days, columns=["event"])

In [835]:
last_15days = last_15days.groupby("person",as_index=False).sum()
last_15days = last_15days[["person","count_activity","event_conversion","event_checkout","event_viewed product"]]


In [836]:
last_15days.columns = ["person","count_activity","conversiones_15lastdays","checkout_15lastdays","viewed_15lastday"]

Nuevo formateo de timedata

In [837]:
timeData["minute"] = timeData["timestamp"].map(lambda x: x.minute)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Se elimina el year por ser todos 2018

In [838]:
timeData["year"].value_counts()

2018    2341681
Name: year, dtype: int64

In [839]:
timeData = timeData.drop("year",axis = 1)

Nuevo formateo de TimeData

In [840]:
timeData = pd.get_dummies(timeData, columns=["event"])

In [841]:
timeData = timeData.drop(["timestamp","Parte del dia"], axis=1)

In [842]:
timeData = timeData.drop("minute",axis=1)

Feature con sesiones de los usuarios

In [843]:
secionesUsuarios = timeData.groupby(["person","month","day","hour"]).sum()

In [844]:
secionesUsuarios["cant_seciones"] = 1

In [845]:
secionesUsuarios = secionesUsuarios[["cant_seciones","event_checkout","event_conversion","event_ad campaign hit","event_lead"]]


In [846]:
secionesUsuarios = secionesUsuarios.reset_index()

In [847]:
secionesUsuarios = secionesUsuarios.drop(["day","hour","month"],axis=1)

In [848]:
secionesUsuarios = secionesUsuarios.groupby(["person"]).sum().fillna(0)

In [849]:
secionesUsuarios = secionesUsuarios.reset_index()

Por dispositivo que usa el usuario para entrar al sitio

In [850]:
deviceUsersData["operating_system_version"].map(lambda x: str(x).split(" ")[0]).value_counts()

Windows       1262240
Android        975740
iOS             73279
Mac             10206
Linux            9456
Ubuntu           3313
nan              2870
Chrome           2376
Other            1048
Tizen             531
BlackBerry        518
Fedora             92
FreeBSD             8
Symbian             4
Name: operating_system_version, dtype: int64

In [851]:
deviceUsersData["operating_system_version"] = deviceUsersData["operating_system_version"].map(lambda x: str(x).split(" ")[0])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [852]:
deviceUsersData["device_type"] = deviceUsersData["device_type"].map(lambda x: "otros" if(x == "Tablet" or x=="Unknown") else x)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [853]:
device_user_type = deviceUsersData[["person","device_type"]]

In [854]:
device_user_type["count"] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [855]:
device_user_type = device_user_type.groupby(["person","device_type"]).max().unstack().fillna(0).reset_index()

In [856]:
multiindex = device_user_type.columns
multiindex = multiindex.tolist()
multiindex
ind = pd.Index([ (e[1] if e[0]!= "person" else e[0]) for e in multiindex])
device_user_type.columns = ind


In [857]:
device_user_system = deviceUsersData[["person","operating_system_version"]]

In [858]:
device_user_system["count"] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [859]:
device_user_system = device_user_system.groupby(["person","operating_system_version"]).max().unstack().fillna(0).reset_index()

In [860]:
multiindex = device_user_system.columns
multiindex = multiindex.tolist()
multiindex
ind = pd.Index([ (e[1] if e[0]!= "person" else e[0]) for e in multiindex])
device_user_system.columns = ind

In [861]:
device_user_system = device_user_system[["person","Windows","Android","iOS"]]

Informacion usuarios

In [862]:
dic_terminos = {}
for busqueda in infoUserData["search_term"]:
    for term in str(busqueda).split(" "):
        if (not term in dic_terminos.keys()):
            dic_terminos[term] = 1
        else:
            dic_terminos[term]+=1     

In [863]:
list_terminos = sorted(dic_terminos.items(), key=lambda kv: kv[1])

In [864]:
list_terminos = list_terminos[::-1]

In [865]:
list_terminos.pop(0)

('nan', 2227918)

In [866]:
terminos = []
for name,count in list_terminos:
    if(len(name)>2):
        terminos.append((name,count))    

In [867]:
terminos_populares = terminos[:300]

In [868]:
infoUserData["search_term"] = infoUserData["search_term"].fillna("0")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [869]:
def verificarPopularidadElTermino(string,terminospopulares):
    if (string==0):
        return False
    for palabra in string.split(" "):
        
        if(palabra in [name for name,count in terminospopulares]):
            return True
        
    return False

In [870]:
infoUserData["Busca_terminos_populares"] = infoUserData["search_term"].map(lambda x:1 if(verificarPopularidadElTermino(x,terminos_populares)) else 0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Feature con el campaign_source y channel mas populares

In [871]:
person_campaing = infoUserData[["person","campaign_source"]]
person_channel = infoUserData[["person","channel"]]

In [872]:
person_term = infoUserData[["person","Busca_terminos_populares"]]

In [873]:
person_term = person_term.groupby("person",as_index = False).sum()

In [874]:
person_campaing = person_campaing.dropna()

In [875]:
person_campaing = pd.get_dummies(person_campaing, columns=["campaign_source"])

In [876]:
person_campaing = person_campaing[["person","campaign_source_google","campaign_source_criteo","campaign_source_rtbhouse"]]

In [877]:
person_campaing = person_campaing.groupby("person",as_index=False).sum()

In [878]:
person_channel = person_channel.dropna()

In [879]:
person_channel = pd.get_dummies(person_channel, columns=["channel"])

In [880]:
person_channel = person_channel[["person","channel_Direct","channel_Paid","channel_Organic","channel_Referral"]]

In [881]:
person_channel = person_channel.groupby("person",as_index=False).sum()

Productos visto por el usuario

In [882]:
productData["model"] = productData["model"].fillna("Desconocido")
productData["marca"] = productData["model"].map(lambda x:"apple" if(x.split(" ")[0] in ["iphone","ipad"]) else x.split(" ")[0])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [883]:
productData["version"] = productData["model"].map(lambda x:" ".join(x.split(" ")[0:2]) if(x.split(" ")[0] in ["iphone","ipad"]) else " ".join(x.split(" ")[1:3]))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [884]:
users_view_list = productData[["person","skus"]]

In [885]:
productData = productData[(productData["event"] == "checkout")]

In [886]:
productData = productData[["person","marca","condition","version","storage"]]

In [887]:
productData= pd.get_dummies(productData, columns=["marca","condition","version","storage"])
productDataDummies = productData

In [888]:
productData = productData.groupby("person",as_index=False).sum()

In [889]:
productData = productData.drop(["marca_quantum","condition_Novo"],axis = 1)

In [890]:
productData = productData[['person', 'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom',
       'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom',"version_galaxy j7","version_galaxy j5","version_galaxy s7","version_iphone 6","version_iphone 5s","version_iphone 6s","version_moto g4","version_moto g2","version_galaxy s6",'storage_128GB', 'storage_16GB', 'storage_256GB', 'storage_32GB',
       'storage_4GB', 'storage_512MB', 'storage_64GB', 'storage_8GB']]

In [891]:
users_view_list["skus"] = users_view_list["skus"].fillna(0)

In [892]:
users_view_list["skus"] = users_view_list["skus"].map(lambda x:0 if(x == 0) else 1)

In [893]:
users_view_list = users_view_list.groupby("person",as_index=False).max()

Feature de new vs returning?

In [894]:
new_last15days = infoUserData[["person","timestamp","new_vs_returning"]]

In [895]:
new_last15days["timestamp"]= datetime(new_last15days["timestamp"],"%Y-%m-%dT%H:%M:%S")
new_last15days["month"] = new_last15days["timestamp"].map(lambda x: x.month)
new_last15days["day"] = new_last15days["timestamp"].map(lambda x: x.day)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [896]:
new_last15days = new_last15days[new_last15days["month"] == 5]
new_last15days = new_last15days[new_last15days["day"] >14]

In [897]:
new_last15days["new_15lastdays"] = new_last15days["new_vs_returning"] == "New"

In [898]:
new_last15days["new_15lastdays"].value_counts()

False    1377983
True       25484
Name: new_15lastdays, dtype: int64

In [899]:
new_last15days = new_last15days[["person","new_15lastdays"]]


In [900]:
new_last15days = new_last15days.groupby("person",as_index=False).max()

In [901]:
new_last15days["new_15lastdays"] = new_last15days["new_15lastdays"].map(lambda x: 1 if(x) else 0) 

Features geograficos

In [902]:
geoData = geoData[geoData["event"] == "conversion"]

In [903]:
region = geoData[["person","region"]]

In [904]:
region = region.drop_duplicates(subset = "person", keep = "first")

In [905]:
(region["region"].value_counts() > 50).value_counts()

False    29
True     15
Name: region, dtype: int64

In [906]:
region["region"] = region["region"].fillna(0)

In [907]:
sell_region = region["region"].value_counts()

In [908]:
region["ventas_region"] = region["region"].map(lambda x: sell_region[x]) 

In [909]:
esBrasilero = geoData[["person","country"]]

In [910]:
esBrasilero["Brasilero"] = esBrasilero["country"].map(lambda x: 1 if(x == "Brazil") else 0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [911]:
esBrasilero = esBrasilero[["person","Brasilero"]]

In [912]:
esBrasilero = esBrasilero.groupby("person",as_index=False).max()

Feature con Colores

In [913]:
personColor["color"].value_counts().head()

Preto             314925
Dourado           275212
Cinza espacial    151770
Branco            126496
Prateado          114284
Name: color, dtype: int64

In [914]:
personColor = personColor.dropna()

In [915]:
bestcolorseller = personColor[personColor["event"] == "checkout"]
colorsellers = bestcolorseller["color"].value_counts()

In [916]:
personColor = pd.get_dummies(personColor, columns=["color"])

In [917]:
personColor = personColor[personColor["event"] == "viewed product"]

In [918]:
personColor = personColor[["person","color_Preto","color_Dourado","color_Branco","color_Cinza espacial"]]


In [919]:
personColor = personColor.groupby("person").sum()

Feature con la probabilidad de compra

In [920]:
versiones = productDataDummies

#65315 son las cantidades de conversiones
versiones['marca_apple'] = (versiones['marca_apple'].sum())/65315
versiones['marca_asus'] = (versiones['marca_asus'].sum())/65315
versiones['marca_lenovo'] = (versiones['marca_lenovo'].sum())/65315
versiones['marca_lg']= (versiones['marca_lg'].sum())/65315
versiones['marca_motorola']= (versiones['marca_motorola'].sum())/65315
versiones['marca_quantum']= (versiones['marca_quantum'].sum())/65315
versiones['marca_samsung']= (versiones['marca_samsung'].sum())/65315
versiones['marca_sony']= (versiones['marca_sony'].sum())/65315
#Aca se puede ver la probabilidad de conversion de cada marca
#Yo quiero la probabilidad de cada usuario de que compre una marca
#Necesito la cantidad de compras de cada usuario por marca

In [921]:
comprasPorMarcaData = eventsData[["person","event","model"]]
comprasPorMarcaData = comprasPorMarcaData[comprasPorMarcaData['event'].map(lambda x: x =='checkout')]
comprasPorMarcaData["model"] = comprasPorMarcaData["model"].fillna("Desconocido")
comprasPorMarcaData["marca"] = comprasPorMarcaData["model"].map(lambda x:"apple" if(x.split(" ")[0] in ["iphone","ipad"]) else x.split(" ")[0])
comprasPorMarcaData = pd.get_dummies(comprasPorMarcaData, columns = ['marca'])
comprasPorMarcaData['marca_apple'] = comprasPorMarcaData['marca_apple'].map(lambda x: x* 0.409875)
comprasPorMarcaData['marca_asus'] = comprasPorMarcaData['marca_asus'].map(lambda x: x* 0.001562)
comprasPorMarcaData['marca_lenovo'] = comprasPorMarcaData['marca_lenovo'].map(lambda x: x* 0.008773)
comprasPorMarcaData['marca_lg'] = comprasPorMarcaData['marca_lg'].map(lambda x: x* 0.017454)
comprasPorMarcaData['marca_motorola'] = comprasPorMarcaData['marca_motorola'].map(lambda x: x* 0.129251)
comprasPorMarcaData['marca_quantum'] = comprasPorMarcaData['marca_quantum'].map(lambda x: x* 0.000505)
comprasPorMarcaData['marca_samsung'] = comprasPorMarcaData['marca_samsung'].map(lambda x: x* 0.426211)
comprasPorMarcaData['marca_sony'] = comprasPorMarcaData['marca_sony'].map(lambda x: x* 0.006369)
comprasPorMarcaData = comprasPorMarcaData.groupby('person').agg({'marca_apple':'sum','marca_asus':'sum','marca_lenovo':'sum','marca_lg':'sum','marca_motorola':'sum','marca_quantum':'sum','marca_samsung':'sum','marca_sony':'sum'})
comprasPorMarcaData.columns = ['Prob_Apple','Prob_Asus','Prob_Lenovo','Prob_lg','Prob_motorola','Prob_quantum','Prob_samsung','Prob_sony']

Feature con el screen resolution

In [922]:
resolutionData = eventsData[['person','event','screen_resolution']]
resolutionData = resolutionData[resolutionData['event'].map(lambda x: x == 'conversion')]
resolutionData = resolutionData.fillna('')
resolutionData['screen_resolution'] = resolutionData['screen_resolution'].map(lambda x: 'A' if( x == '360x640') else x)
resolutionData['screen_resolution'] = resolutionData['screen_resolution'].map(lambda x: 'B' if (x == '1366x768')else x)
resolutionData['screen_resolution'] = resolutionData['screen_resolution'].map(lambda x: 'C' if (x == '1920x1080') else x)
resolutionData['screen_resolution'] = resolutionData['screen_resolution'].map(lambda x: 'D' if (x == '1360x768') else x)
resolutionData['screen_resolution'] = resolutionData['screen_resolution'].map(lambda x: 'E' if (x != 'A'and x!= 'B'and x!='C' and x!='D') else x)
resolutionData = pd.get_dummies(resolutionData, columns = ['screen_resolution'])
resolutionData = resolutionData.groupby('person').agg({'screen_resolution_A':'sum','screen_resolution_B':'sum','screen_resolution_C':'sum','screen_resolution_D':'sum','screen_resolution_E':'sum'})

Mergeo todos mis resultados de los nuevos features

In [923]:
secionesUsuariosReducido = secionesUsuarios#[["person","cant_seciones5","cant_seciones4","cant_seciones3","event_checkout5","event_checkout4","event_conversion5","event_conversion4","event_ad campaign hit4","event_ad campaign hit5","event_lead5","event_lead4"]]

In [924]:
dataTrocafone = pd.merge(secionesUsuariosReducido,users_view_list,on="person",how = "outer")
dataTrocafone['person'].count()

38829

In [925]:
dataTrocafone = pd.merge(dataTrocafone,resolutionData,on="person",how = "outer")
dataTrocafone = dataTrocafone.fillna(0)

In [926]:
dataTrocafone = pd.merge(dataTrocafone,person_term,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [927]:
dataTrocafone = pd.merge(dataTrocafone,device_user_type,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [928]:
dataTrocafone["Computer"] = dataTrocafone["Computer"].fillna(0)
dataTrocafone['person'].count()

38829

In [929]:
dataTrocafone["Smartphone"] = dataTrocafone["Smartphone"].fillna(0)
dataTrocafone['person'].count()

38829

In [930]:
dataTrocafone["otros"] = dataTrocafone["otros"].fillna(0)
dataTrocafone['person'].count()

38829

In [931]:
dataTrocafone = pd.merge(dataTrocafone,productData,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [932]:
dataTrocafone = pd.merge(dataTrocafone,users_partedeldia,on="person",how="left")
dataTrocafone['person'].count()

38829

In [933]:
dataTrocafone = pd.merge(dataTrocafone,person_campaing,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [934]:
dataTrocafone = pd.merge(dataTrocafone,person_channel,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [935]:
dataTrocafone = pd.merge(dataTrocafone,esBrasilero,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [936]:
dataTrocafone = dataTrocafone.fillna(0)

In [937]:
dataTrocafone = pd.merge(dataTrocafone,region,on="person",how="outer")
dataTrocafone['person'].count()

38829

In [938]:
dataTrocafone = dataTrocafone.drop("region",axis=1)

In [939]:
dataTrocafone["ventas_region"] = dataTrocafone["ventas_region"].fillna(0)

In [940]:
dataTrocafone = pd.merge(dataTrocafone,last_15days,how = "left", on="person")
dataTrocafone['person'].count()

38829

In [941]:
dataTrocafone["count_activity"] = dataTrocafone["count_activity"].fillna(0)

In [942]:
dataTrocafone["conversiones_15lastdays"] = dataTrocafone["conversiones_15lastdays"].fillna(0)
dataTrocafone["checkout_15lastdays"] = dataTrocafone["checkout_15lastdays"].fillna(0)
dataTrocafone["viewed_15lastday"] = dataTrocafone["viewed_15lastday"].fillna(0)

In [943]:
personColor = personColor.reset_index()

In [944]:
dataTrocafone = pd.merge(dataTrocafone,personColor,how = "left", on="person")
dataTrocafone['person'].count()

38829

In [945]:
dataTrocafone["color_Preto"] = dataTrocafone["color_Preto"].fillna(0)
dataTrocafone["color_Dourado"] = dataTrocafone["color_Dourado"].fillna(0)
dataTrocafone["color_Branco"] = dataTrocafone["color_Branco"].fillna(0)
dataTrocafone["color_Cinza espacial"] = dataTrocafone["color_Cinza espacial"].fillna(0)

In [946]:
dataTrocafone = pd.merge(dataTrocafone,new_last15days,how = "left", on="person")
dataTrocafone['person'].count()

38829

In [947]:
dataTrocafone["new_15lastdays"] = dataTrocafone["new_15lastdays"].fillna(0)


In [948]:
dataTrocafone = pd.merge(dataTrocafone,skuData,how = "left", on="person")
dataTrocafone = pd.merge(dataTrocafone,dataSearchAndSelling,how = "left", on="person")
dataTrocafone['person'].count()

38829

In [949]:
dataTrocafone["cantidad_De_Compras_De_Mismo_Sku"] = dataTrocafone["cantidad_De_Compras_De_Mismo_Sku"].fillna(0)
dataTrocafone["sku"] = dataTrocafone["sku"].fillna(0)
dataTrocafone['person'].count()

38829

In [950]:
dataTrocafone["search_and_purchase"] = dataTrocafone["search_and_purchase"].fillna(0)
dataTrocafone["Cantidad_De_Busquedas_Por_Termino"] = dataTrocafone["Cantidad_De_Busquedas_Por_Termino"].fillna(0)
dataTrocafone['person'].count()

38829

In [951]:
dataTrocafone = pd.merge(dataTrocafone,auxiliar,how = "left", on="person")
dataTrocafone["cantidad_de_Url_de_compra"] = dataTrocafone["cantidad_de_Url_de_compra"].fillna(0)
dataTrocafone['person'].count()

38829

In [952]:
dataTrocafone = pd.merge(dataTrocafone,staticPageData,how = "left", on="person")
dataTrocafone["cantidadDeStaticPage"] = dataTrocafone["cantidadDeStaticPage"].fillna(0)
dataTrocafone['person'].count()

38829

In [953]:
dataTrocafone = pd.merge(dataTrocafone,dataF,how = "left", on="person")
dataTrocafone["sku_multiple"] = dataTrocafone["sku_multiple"].fillna(0)
dataTrocafone.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 38828
Data columns (total 73 columns):
person                               38829 non-null object
cant_seciones                        38829 non-null int64
event_checkout                       38829 non-null float64
event_conversion                     38829 non-null float64
event_ad campaign hit                38829 non-null float64
event_lead                           38829 non-null float64
skus                                 38829 non-null int64
screen_resolution_A                  38829 non-null float64
screen_resolution_B                  38829 non-null float64
screen_resolution_C                  38829 non-null float64
screen_resolution_D                  38829 non-null float64
screen_resolution_E                  38829 non-null float64
Busca_terminos_populares             38829 non-null int64
Computer                             38829 non-null float64
Smartphone                           38829 non-null float6

Armo el set de training y el de test

In [954]:
trainingSet = pd.read_csv("data/labels_training_set.csv", low_memory = False)

In [955]:
trainingSet["person"].count()

19414

In [956]:
dataTrocafone["person"].count()

38829

In [957]:
setTraining = pd.merge(dataTrocafone,trainingSet,on="person",how = "inner")

In [958]:
setTraining["person"].count()

19414

In [959]:
setTraining.to_csv('setEntrenamiento.csv')

In [960]:
kaggle_test = pd.read_csv("data/trocafone_kaggle_test.csv", low_memory = False)

In [961]:
kaggle_test["person"].count()

19415

In [962]:
test = pd.merge(kaggle_test,dataTrocafone,on="person",how="inner")

In [963]:
test["person"].count()

19415

In [964]:
test.to_csv("setTest.csv")

In [965]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19415 entries, 0 to 19414
Data columns (total 73 columns):
person                               19415 non-null object
cant_seciones                        19415 non-null int64
event_checkout                       19415 non-null float64
event_conversion                     19415 non-null float64
event_ad campaign hit                19415 non-null float64
event_lead                           19415 non-null float64
skus                                 19415 non-null int64
screen_resolution_A                  19415 non-null float64
screen_resolution_B                  19415 non-null float64
screen_resolution_C                  19415 non-null float64
screen_resolution_D                  19415 non-null float64
screen_resolution_E                  19415 non-null float64
Busca_terminos_populares             19415 non-null int64
Computer                             19415 non-null float64
Smartphone                           19415 non-null float6